In [1]:
from test import test_model

In [2]:
metrics, pred, labels = test_model('./test_config.yaml', return_pred=True)

WrappedUnetPlusPlus: <All keys matched successfully>


100%|██████████| 196/196 [00:00<00:00, 1248.90it/s]


Warning! dataset/sessile-main-Kvasir-SEG/kavsir_bboxes.json file not found, dataset will be without bboxes


getting predictions: 100%|██████████| 196/196 [00:57<00:00,  3.44it/s]


In [4]:
from tabulate import tabulate

table = [[k, f'{v:.3}'] for k, v in metrics.items()]
print(tabulate(table))

------------  -----
dice_group_0  0.924
iou_group_0   0.863
dice_group_1  0.967
iou_group_1   0.935
dice_group_2  0.97
iou_group_2   0.942
dice_group_3  0.969
iou_group_3   0.941
dice_group_4  0.979
iou_group_4   0.959
dice          0.933
iou           0.878
------------  -----


In [ ]:
from PIL import Image
from torchvision.transforms import ToPILImage
import torch.nn.functional as F
to_pill_image = ToPILImage()

In [ ]:
from src.data_utils.dataset import load_dataset

test_dataset = load_dataset(r'./test_config.yaml')

In [ ]:
idx = 42

logits = pred[idx]
target_mask = labels[idx]

logits = F.softmax(logits, dim=0)
pred_mask = logits.argmax(dim=0)

pred_mask_tensor = pred_mask.bool()

pred_mask = to_pill_image(pred_mask.float())
target_mask = to_pill_image(target_mask.float())

# pred_mask.show()
# target_mask.show()

In [ ]:
from src.data_utils.utils import plot_mask_over_image

data_ = test_dataset.__getitem__(idx)
input_ = data_['input']
target_ = data_['target'].bool()

plot_mask_over_image(input_, target_)
plot_mask_over_image(input_, pred_mask_tensor)

In [ ]:
from train import train_setup

In [ ]:
trainer = train_setup(r'./train_config.yaml')

In [ ]:
# set(trainer.train_dataset.data_names).intersection(set(trainer.eval_dataset.data_names))

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
print(trainer.train_dataset.dataset[0]['input'].shape)
trainer.train_dataset.dataset[0]['target'].shape

In [ ]:
print(trainer.eval_dataset[0]['input'].shape)
trainer.eval_dataset[0]['target'].shape

In [ ]:
trainer.train_dataset.dataset.augmentator()